# 데이터 로드

In [1]:
import numpy as np
import pandas as pd

combined_review_origin = pd.read_excel('combined_review.xlsx')
combined_review_origin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30555 entries, 0 to 30554
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   회사명     30555 non-null  object        
 1   고유 제품명  30555 non-null  object        
 2   형태      30555 non-null  object        
 3   작성일     30555 non-null  datetime64[ns]
 4   별점      30555 non-null  int64         
 5   제목_본문   30555 non-null  object        
 6   채널명     30555 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 1.6+ MB


In [2]:
# 카피본 떠놓고 시작하기
combined_review = combined_review_origin.copy()
combined_review.shape

(30555, 7)

In [3]:
combined_review.duplicated().sum()

3

In [4]:
combined_review[['제목_본문']].duplicated().sum()

62

In [5]:
dup_text = combined_review[combined_review[['제목_본문']].duplicated()]['제목_본문'].unique()

    
text = []
idx = []
for d_t in dup_text:
    df1 = combined_review[combined_review['제목_본문']==d_t]
    print(df1)

            회사명       고유 제품명 형태        작성일  별점 제목_본문     채널명
2874  bidetmate  3000 Series  E 2022-09-30   5  Good  amazon
3539   brondell        S1400  E 2022-05-24   5  Good  amazon
9747   brondell        SE600  E 2018-05-23   4  Good  amazon
           회사명       고유 제품명 형태        작성일  별점 제목_본문     채널명
3503      toto  SW3056 S550  E 2022-06-01   1   결측치  amazon
3728  brondell        S1400  E 2022-04-15   5   결측치  amazon
6813  biobidet        BB600  E 2020-10-22   5   결측치  amazon
              회사명       고유 제품명   형태        작성일  별점  제목_본문     채널명
3245         toto  SW3036 K300    E 2022-07-26   5  Great  amazon
4698        tushy    Basic 2.0    A 2021-10-21   5  Great  amazon
5356     biobidet        BB600    E 2021-06-10   5  Great  amazon
6583     biobidet        BB600    E 2020-12-01   5  Great  amazon
8619     biobidet       BB2000    R 2019-08-04   5  Great  amazon
9309   alphabidet     Alpha JX    R 2018-12-20   5  Great  amazon
25875    brondell       CL1500  R E 2022-03-04   5  Gr

# 데이터 전처리

## 긍정/부정 분류
- 평점 5점 -> label == 2
- 평점 1,2점 -> label ==1

In [6]:
combined_review.groupby('별점')[['제목_본문']].count()

,제목_본문
별점,
1,1782
2,1049
3,1624
4,4789
5,21311


In [7]:
bidet_positive = combined_review[combined_review['별점']==5][['별점', '제목_본문']]
bidet_negative = combined_review[combined_review['별점']<=2][['별점', '제목_본문']]
print(bidet_positive[-5:])
print(bidet_negative[-7:])

       별점                                              제목_본문
30547   5  Works well.Worth noting that it works best whe...
30550   5  Worth every penny.We love our new toilet! We h...
30551   5  Worth every penny!.It’s a beautiful toilet tha...
30553   5  You will miss this.Good but needs some tweakin...
30554   5  You’re mad if you didn’t buy this... :).Easy i...
       별점                                              제목_본문
30492   1  Stopped working 1.5 years in. No help from mfg...
30495   1  Terrible Customer Service OVE!.Worked fine for...
30500   2  The functionality of the toilets was too diffi...
30507   1  Toilet does not flush.My toilet does not flush...
30514   1  Useless.We have hardly used it for 2 months an...
30527   2  Water Hammer.After a lengthy installation my p...
30549   1  Worst product.I just opened the box and parts ...


In [8]:
bidet_positive['별점'] = 1
bidet_negative['별점'] = 0
print(bidet_positive[-5:])
print(bidet_negative[-7:])

       별점                                              제목_본문
30547   1  Works well.Worth noting that it works best whe...
30550   1  Worth every penny.We love our new toilet! We h...
30551   1  Worth every penny!.It’s a beautiful toilet tha...
30553   1  You will miss this.Good but needs some tweakin...
30554   1  You’re mad if you didn’t buy this... :).Easy i...
       별점                                              제목_본문
30492   0  Stopped working 1.5 years in. No help from mfg...
30495   0  Terrible Customer Service OVE!.Worked fine for...
30500   0  The functionality of the toilets was too diffi...
30507   0  Toilet does not flush.My toilet does not flush...
30514   0  Useless.We have hardly used it for 2 months an...
30527   0  Water Hammer.After a lengthy installation my p...
30549   0  Worst product.I just opened the box and parts ...


## 컬럼명 변경

In [9]:
bidet_positive.rename(columns={'별점':'label', '제목_본문':'text'}, inplace=True)
bidet_negative.rename(columns={'별점':'label', '제목_본문':'text'}, inplace=True)
print(bidet_positive[-5:])
print(bidet_negative[-7:])

       label                                               text
30547      1  Works well.Worth noting that it works best whe...
30550      1  Worth every penny.We love our new toilet! We h...
30551      1  Worth every penny!.It’s a beautiful toilet tha...
30553      1  You will miss this.Good but needs some tweakin...
30554      1  You’re mad if you didn’t buy this... :).Easy i...
       label                                               text
30492      0  Stopped working 1.5 years in. No help from mfg...
30495      0  Terrible Customer Service OVE!.Worked fine for...
30500      0  The functionality of the toilets was too diffi...
30507      0  Toilet does not flush.My toilet does not flush...
30514      0  Useless.We have hardly used it for 2 months an...
30527      0  Water Hammer.After a lengthy installation my p...
30549      0  Worst product.I just opened the box and parts ...


In [10]:
bidet_positive.shape

(21311, 2)

In [11]:
bidet_negative.shape

(2831, 2)

## 중복 제거

In [12]:
print(bidet_positive.duplicated().sum())
print(bidet_negative.duplicated().sum())

52
0


In [13]:
print(bidet_positive['text'].duplicated().sum())
print(bidet_negative['text'].duplicated().sum())

52
0


In [14]:
bidet_positive[bidet_positive.duplicated()]

,label,text
3539,1,Good
4698,1,Great
5356,1,Great
5709,1,good
6232,1,Easy to install
6583,1,Great
6813,1,결측치
7857,1,Great purchase
8619,1,Great
8809,1,As advertised


In [15]:
bidet_positive.drop_duplicates(inplace=True)
print(bidet_positive.duplicated().sum())
print(bidet_negative.duplicated().sum())

0
0


In [16]:
print(bidet_positive['text'].duplicated().sum())
print(bidet_negative['text'].duplicated().sum())

0
0


## 랜덤 샘플링
- 훈련용 데이터프레임의 인덱스를 테스트용 데이터프레임의 인덱스와 중복되지 않게 설정

In [17]:
# 각각 만 개와 천 개의 샘플링 데이터 추출
train_pos_random = bidet_positive.sample(n=10000, random_state=2502)
train_neg_random = bidet_negative.sample(n=1000, random_state=2502)
print(train_pos_random[-5:])
print(train_neg_random[-5:])

       label                                               text
14921      1  Great product.Great product, easy to install a...
27827      1  Brondell CS1000.Just bought Brondell CS1000, a...
25168      1  Wonderful product.The bidet is easy to install...
7714       1  Very classy.After much research and speaking w...
11218      1  So many features!.I received this bidet as a C...
       label                                               text
3535       0  Not a universal fit.Note that although this pr...
3164       0  Leaks after several attempts to correct.Instal...
13836      0  cheaply made.arrived damaged. was promptly ret...
23335      0  Not an adequate spray. Sloping seat..One singl...
4127       0  1 Year Later….I ordered my Tushy Classic off t...


In [18]:
train_pos_random.shape

(10000, 2)

In [19]:
train_neg_random.shape

(1000, 2)

In [20]:
test_pos_random = bidet_positive.drop(train_pos_random.index)
test_neg_random = bidet_negative.drop(train_neg_random.index)
print(test_pos_random[-5:])
print(test_neg_random[-5:])

       label                                               text
30537      1  Works beautifully.This is a lovely bidet toile...
30547      1  Works well.Worth noting that it works best whe...
30550      1  Worth every penny.We love our new toilet! We h...
30553      1  You will miss this.Good but needs some tweakin...
30554      1  You’re mad if you didn’t buy this... :).Easy i...
       label                                               text
30470      0  Response from Costco Wholesale:.Had purchased ...
30492      0  Stopped working 1.5 years in. No help from mfg...
30495      0  Terrible Customer Service OVE!.Worked fine for...
30507      0  Toilet does not flush.My toilet does not flush...
30549      0  Worst product.I just opened the box and parts ...


In [21]:
test_pos_random.shape

(11259, 2)

In [22]:
test_neg_random.shape

(1831, 2)

## 긍정-부정 병합

In [23]:
# 무작위로 데이터 병합
train_random = pd.concat([train_pos_random, train_neg_random], ignore_index=True).sample(frac=1, random_state=2502)
train_random.head()

,label,text
5546,1,"Once You've Tried, You'll Never Go Back!!.This..."
705,1,"Love it, easy to use, warm seat and lots of be..."
2301,1,Simple to operate simple to install..Simple an...
8578,1,good item.easy to install
10265,0,BioBidet Broke Right After Warranty Expiration...


In [24]:
test_random = pd.concat([test_pos_random, test_neg_random], ignore_index=True).sample(frac=1, random_state=2502)
test_random.head()

,label,text
2291,1,Gives Toto a run for your money..Excellent sol...
3790,1,Amazing.After owning the Bio Bidet. I can’t se...
10996,1,Warning - looong review.Installation
8491,1,Life changing!.We originally purchased this un...
993,1,A wonderful addition to our house! .I have ...


In [25]:
train_random.shape

(11000, 2)

In [26]:
test_random.shape

(13090, 2)

# 조회

In [27]:
train_random.duplicated().sum()

0

In [28]:
test_random.duplicated().sum()

0

In [29]:
train_random.isna().sum()

label    0
text     0
dtype: int64

In [30]:
test_random.isna().sum()

label    0
text     0
dtype: int64

# 파일 저장

In [31]:
train_random.to_excel('bidet_train_data.xlsx', index=False)

In [32]:
test_random.to_excel('bidet_test_data.xlsx', index=False)